In [1]:
pip install geopandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geopy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install gdal

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Using cached GDAL-3.8.3.tar.gz (802 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for gdal
Failed to build gdal


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [117 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-311
  creating build\lib.win-amd64-cpython-311\osgeo
  copying osgeo\gdal.py -> build\lib.win-amd64-cpython-311\osgeo
  copying osgeo\gdalconst.py -> build\lib.win-amd64-cpython-311\osgeo
  copying osgeo\gdalnumeric.py -> build\lib.win-amd64-cpython-311\osgeo
  copying osgeo\gdal_array.py -> build\lib.win-amd64-cpython-311\osgeo
  copying osgeo\gnm.py -> build\lib.win-amd64-cpython-311\osgeo
  copying osgeo\ogr.py -> build\lib.win-amd64-cpython-311\osgeo
  copying osgeo\osr.py -> build\lib.win-amd64-cpython-311\osgeo
  copying osgeo\__init__.py -> build\lib.win-amd64-cpython-311\osgeo
  creating build\lib.win-amd64-cpython-311\osgeo_utils
  copying gdal-utils\osgeo_utils\gdal2tiles.py -> build\lib.win-amd64-cpython-311\osgeo_utils
  copyi

In [4]:
pip install yellowbrick

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [116]:
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from geopy.geocoders import Nominatim
import re
import json
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [117]:
# Menentukan path folder yang berisi dataset COVID-19 Jakarta
folder_path = os.path.join(os.path.expanduser("~"), "Downloads", "covid_dashboard_jakarta_2022-master", "Data_baru")

# Memeriksa apakah folder tersebut ada
if os.path.exists(folder_path):
    # Jika ada, mencetak nama semua file dalam folder
    for file in os.listdir(folder_path):
        print(file)
else:
    # Jika tidak ada, mencetak pesan bahwa folder tidak ditemukan
    print("Folder tidak ditemukan.")

Standar Kelurahan Data Corona (01 Agustus 2022 pukul 15.00).xlsx
Standar Kelurahan Data Corona (01 April 2022 Pukul 15.00).xlsx
Standar Kelurahan Data Corona (01 Desember 2021 Pukul 10.00).xlsx
Standar Kelurahan Data Corona (01 Desember 2022 pukul 15.00).xlsx
Standar Kelurahan Data Corona (01 Februari 2022 Pukul 15.00).xlsx
Standar Kelurahan Data Corona (01 Januari 2022 Pukul 10.00).xlsx
Standar Kelurahan Data Corona (01 Januari 2023 pukul 15.00).xlsx
Standar Kelurahan Data Corona (01 Juli 2022 pukul 15.00).xlsx
Standar Kelurahan Data Corona (01 Juni 2022 pukul 15.00).xlsx
Standar Kelurahan Data Corona (01 Maret 2022 Pukul 15.00).xlsx
Standar Kelurahan Data Corona (01 Mei 2022 pukul 15.00).xlsx
Standar Kelurahan Data Corona (01 November 2021 Pukul 10.00).xlsx
Standar Kelurahan Data Corona (01 November 2022 pukul 15.00).xlsx
Standar Kelurahan Data Corona (01 Oktober 2021 Pukul 10.00).xlsx
Standar Kelurahan Data Corona (01 Oktober 2022 pukul 15.00).xlsx
Standar Kelurahan Data Corona (01 

In [118]:
# Menentukan path direktori utama untuk dataset COVID-19 Jakarta
main_directory = os.path.join(os.path.expanduser("~"), "Downloads", "covid_dashboard_jakarta_2022-master")

# Gabungkan dengan nama subdirektori "Original Data Sets"
path = os.path.join(main_directory, "Data_baru")

# Inisialisasi list untuk menyimpan dataframe dari setiap file Excel
df_list = []

# Iterasi melalui setiap file dalam subdirektori
for file in os.listdir(path):
    # Memeriksa apakah file berakhir dengan ".xlsx"
    if file.endswith(".xlsx"):
        # Membaca file Excel dan menambahkan kolom "Date" dengan nilai nama file
        df = pd.read_excel(os.path.join(path, file))
        df["Date"] = file
        df_list.append(df)

# Menggabungkan semua dataframe menjadi satu dataframe besar
df = pd.concat(df_list, axis=0)

# Cleaning Data

In [119]:
# Menghapus baris yang memiliki nilai "TOTAL" pada kolom "nama_kelurahan"
df = df[df["nama_kelurahan"] != "TOTAL"]

# Mengatur ulang indeks dataframe setelah penghapusan
df = df.reset_index(drop=True)

# Menampilkan 100 baris pertama dari dataframe
df.head(100)

,ID_KEL,Nama_provinsi,nama_kota,nama_kecamatan,nama_kelurahan,SUSPEK,Perawatan RS,Isolasi di Rumah,Suspek Meninggal,Selesai Isolasi,...,Isolasi di Rumah.4,Meninggal,Selesai Isolasi.4,POSITIF,Dirawat,Sembuh,Meninggal.1,Self Isolation,Keterangan,Date
0,3172051003,DKI JAKARTA,JAKARTA UTARA,PADEMANGAN,ANCOL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2945,7,2875,30,33,NaN,Standar Kelurahan Data Corona (01 Agustus 2022...
1,3173041007,DKI JAKARTA,JAKARTA BARAT,TAMBORA,ANGKE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2870,2,2790,35,43,NaN,Standar Kelurahan Data Corona (01 Agustus 2022...
2,3175041005,DKI JAKARTA,JAKARTA TIMUR,KRAMAT JATI,BALE KAMBANG,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3196,1,3131,36,28,NaN,Standar Kelurahan Data Corona (01 Agustus 2022...
3,3175031003,DKI JAKARTA,JAKARTA TIMUR,JATINEGARA,BALI MESTER,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1361,2,1315,20,24,NaN,Standar Kelurahan Data Corona (01 Agustus 2022...
4,3175101006,DKI JAKARTA,JAKARTA TIMUR,CIPAYUNG,BAMBU APUS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4266,6,4167,36,57,NaN,Standar Kelurahan Data Corona (01 Agustus 2022...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3171021002,DKI JAKARTA,JAKARTA PUSAT,SAWAH BESAR,KARANG ANYAR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2982,3,2911,42,26,NaN,Standar Kelurahan Data Corona (01 Agustus 2022...
96,3174021004,DKI JAKARTA,JAKARTA SELATAN,SETIA BUDI,KARET,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1675,0,1641,13,21,NaN,Standar Kelurahan Data Corona (01 Agustus 2022...
97,3174021003,DKI JAKARTA,JAKARTA SELATAN,SETIA BUDI,KARET KUNINGAN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4480,0,4378,47,55,NaN,Standar Kelurahan Data Corona (01 Agustus 2022...
98,3174021002,DKI JAKARTA,JAKARTA SELATAN,SETIA BUDI,KARET SEMANGGI,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,727,0,707,6,14,NaN,Standar Kelurahan Data Corona (01 Agustus 2022...


In [120]:
#@ title info type data
print('Tipe Data Sebelum:\n')
df.info()

Tipe Data Sebelum:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133155 entries, 0 to 133154
Data columns (total 34 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ID_KEL              133155 non-null  object 
 1   Nama_provinsi       133155 non-null  object 
 2   nama_kota           133155 non-null  object 
 3   nama_kecamatan      133155 non-null  object 
 4   nama_kelurahan      133155 non-null  object 
 5   SUSPEK              3766 non-null    float64
 6   Perawatan RS        3766 non-null    float64
 7   Isolasi di Rumah    3766 non-null    float64
 8   Suspek Meninggal    3766 non-null    float64
 9   Selesai Isolasi     3766 non-null    float64
 10  PROBABLE            3766 non-null    float64
 11  Perawatan RS.1      3766 non-null    float64
 12  Isolasi di Rumah.1  3766 non-null    float64
 13  Probable Meninggal  3766 non-null    float64
 14  Selesai Isolasi.1   3766 non-null    float64
 15  PELAKU PERJALA

In [121]:
# Mengganti teks pada kolom "Date" untuk memformatnya
df["Date"] = df["Date"].str.replace("Standar Kelurahan Data Corona ", "")
df["Date"] = df["Date"].str.replace(".xlsx", "", regex=True)
df["Date"] = df["Date"].str.replace(re.escape("("), "", regex=True)
df["Date"] = df["Date"].str.replace(re.escape(")"), "", regex=True)
df["Date"] = df["Date"].str.replace("pukul", "")
df["Date"] = df["Date"].str.replace("Pukul", "")
df["Date"] = df["Date"].str.replace(".", ":", regex=True)
df["Date"] = df["Date"].str.replace("  ", " ")

In [122]:
print(df["Date"].unique())

['::::::::::::::::::::::' '::::::::::::::::::::' ':::::::::::::::::::::::'
 ':::::::::::::::::::' '::::::::::::::::::' '::::::::::::::::::::::::']


In [123]:
df.columns

Index(['ID_KEL', 'Nama_provinsi', 'nama_kota', 'nama_kecamatan',
       'nama_kelurahan', 'SUSPEK', 'Perawatan RS', 'Isolasi di Rumah',
       'Suspek Meninggal', 'Selesai Isolasi', 'PROBABLE', 'Perawatan RS.1',
       'Isolasi di Rumah.1', 'Probable Meninggal', 'Selesai Isolasi.1',
       'PELAKU PERJALANAN', 'Perawatan RS.2', 'Isolasi di Rumah.2',
       'Selesai Isolasi.2', 'KONTAK ERAT', 'Perawatan RS.3',
       'Isolasi di Rumah.3', 'Selesai Isolasi.3', 'DISCARDED',
       'Isolasi di Rumah.4', 'Meninggal', 'Selesai Isolasi.4', 'POSITIF',
       'Dirawat', 'Sembuh', 'Meninggal.1', 'Self Isolation', 'Keterangan',
       'Date'],
      dtype='object')

In [124]:
df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d", errors="coerce")
df["Date"] = df["Date"].combine_first(pd.to_datetime(df["Date"], format="%m/%d/%Y", errors="coerce"))

In [125]:
# Pemetaan nama bulan ke bahasa Inggris
month_mapper = {
    "Januari": "January",
    "Februari": "February",
    "Maret": "March",
    "April": "April",
    "Mei": "May",
    "Juni": "June",
    "Juli": "July",
    "Agustus": "August",
    "September": "September",
    "Oktober": "October",
    "November": "November",
    "Desember": "December",
}

# Mengaplikasikan pemetaan nama bulan ke kolom "Date"
df["Date"] = df["Date"].apply(lambda x: " ".join([month_mapper.get(i, i) for i in str(x).split()]) if not pd.isna(x) else x)

# Mengonversi kolom "Date" ke tipe data datetime
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

In [126]:
# Memfilter data untuk tahun 2022
df = df[df["Date"].dt.year == 2022]

# Menghapus baris dengan nama kecamatan "LUAR DKI JAKARTA" dan "PROSES UPDATE DATA"
df = df[df["nama_kecamatan"] != "LUAR DKI JAKARTA"]
df = df[df["nama_kecamatan"] != "PROSES UPDATE DATA"]

# Mengganti teks "KEP." menjadi "KEPULAUAN" pada kolom "nama_kecamatan"
df["nama_kecamatan"] = df["nama_kecamatan"].str.replace("KEP.", "KEPULAUAN", regex=True)

# Mengurutkan dataframe berdasarkan kolom "Date" dan mengatur ulang indeks
df = df.sort_values("Date").reset_index(drop=True)

In [127]:
df.columns

Index(['ID_KEL', 'Nama_provinsi', 'nama_kota', 'nama_kecamatan',
       'nama_kelurahan', 'SUSPEK', 'Perawatan RS', 'Isolasi di Rumah',
       'Suspek Meninggal', 'Selesai Isolasi', 'PROBABLE', 'Perawatan RS.1',
       'Isolasi di Rumah.1', 'Probable Meninggal', 'Selesai Isolasi.1',
       'PELAKU PERJALANAN', 'Perawatan RS.2', 'Isolasi di Rumah.2',
       'Selesai Isolasi.2', 'KONTAK ERAT', 'Perawatan RS.3',
       'Isolasi di Rumah.3', 'Selesai Isolasi.3', 'DISCARDED',
       'Isolasi di Rumah.4', 'Meninggal', 'Selesai Isolasi.4', 'POSITIF',
       'Dirawat', 'Sembuh', 'Meninggal.1', 'Self Isolation', 'Keterangan',
       'Date'],
      dtype='object')

In [128]:
# Menggunakan geolocator untuk mendapatkan koordinat (latitude, longitude) setiap kecamatan
geolocator = Nominatim(user_agent="MyApp")
latitude = {}
longitude = {}

for kecamatan in df["nama_kecamatan"].unique():
    try:
        location = geolocator.geocode(kecamatan, timeout=None)
        latitude[kecamatan] = location.latitude
        longitude[kecamatan] = location.longitude
    except:
        print(f"Error: Tidak dapat menemukan lokasi untuk {kecamatan}")

In [129]:
df["latitude_kecamatan"] = df["nama_kecamatan"].apply(lambda x: latitude.get(x))
df["longitude_kecamatan"] = df["nama_kecamatan"].apply(lambda x: longitude.get(x))

In [130]:
df.to_pickle("combined_data.pkl")

In [131]:
file_path = os.path.join(os.path.expanduser("~"), "Downloads", "covid_dashboard_jakarta_2022-master","combined_data.pkl")
df = pd.read_pickle(file_path)
# Tampilkan lima baris pertama dari DataFrame
df.head()

,nama_provinsi,nama_kota,nama_kecamatan,nama_kelurahan,positif,dirawat,sembuh,meninggal,self_isolation,date,latitude_kecamatan,longitude_kecamatan
0,DKI JAKARTA,JAKARTA UTARA,PADEMANGAN,ANCOL,1947,0,1919,27,1,2022-01-01 10:00:00,-6.129052,106.828972
1,DKI JAKARTA,JAKARTA SELATAN,PASAR MINGGU,PASAR MINGGU,2933,0,2882,51,0,2022-01-01 10:00:00,-6.282674,106.844941
2,DKI JAKARTA,JAKARTA PUSAT,SENEN,PASEBAN,2564,0,2527,37,0,2022-01-01 10:00:00,-6.184971,106.843235
3,DKI JAKARTA,JAKARTA BARAT,KALI DERES,PEGADUNGAN,4466,2,4403,59,2,2022-01-01 10:00:00,-7.235376,110.080258
4,DKI JAKARTA,JAKARTA PUSAT,MENTENG,PEGANGSAAN,1976,0,1927,49,0,2022-01-01 10:00:00,-6.195026,106.832224


In [132]:
df.columns

Index(['nama_provinsi', 'nama_kota', 'nama_kecamatan', 'nama_kelurahan',
       'positif', 'dirawat', 'sembuh', 'meninggal', 'self_isolation', 'date',
       'latitude_kecamatan', 'longitude_kecamatan'],
      dtype='object')

In [133]:
# Misalkan df adalah nama DataFrame Anda
df = df.sort_values(by=['nama_kelurahan', 'date'], ascending=[True, False])

# Menampilkan hasil pengurutan
print(df)


      nama_provinsi      nama_kota     nama_kecamatan nama_kelurahan  positif   
97355   DKI JAKARTA  JAKARTA UTARA         PADEMANGAN          ANCOL     3386  \
97187   DKI JAKARTA  JAKARTA UTARA         PADEMANGAN          ANCOL     3388   
96786   DKI JAKARTA  JAKARTA UTARA         PADEMANGAN          ANCOL     3386   
96552   DKI JAKARTA  JAKARTA UTARA         PADEMANGAN          ANCOL     3385   
96250   DKI JAKARTA  JAKARTA UTARA         PADEMANGAN          ANCOL     3385   
...             ...            ...                ...            ...      ...   
1070    DKI JAKARTA  JAKARTA BARAT  GROGOL PETAMBURAN  WIJAYA KUSUMA     2871   
970     DKI JAKARTA  JAKARTA BARAT  GROGOL PETAMBURAN  WIJAYA KUSUMA     2871   
630     DKI JAKARTA  JAKARTA BARAT  GROGOL PETAMBURAN  WIJAYA KUSUMA     2871   
436     DKI JAKARTA  JAKARTA BARAT  GROGOL PETAMBURAN  WIJAYA KUSUMA     2871   
96      DKI JAKARTA  JAKARTA BARAT  GROGOL PETAMBURAN  WIJAYA KUSUMA     2871   

       dirawat  sembuh  men

In [134]:
df.to_csv('Proyek_2_Data_Cleaning.csv')